In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Pour afficher les graphiques en ligne dans un notebook Jupyter
%matplotlib inline

In [2]:
# Data cleaning
sales_data = pd.read_csv("sales.csv")
sales_df = sales_data.rename(columns={"Unnamed: 0": "uid"})

# X and y determination
X = sales_df.drop(columns=["uid", "date", "open", "sales"])
y = sales_df["sales"]

# Preprocessing
from sklearn import preprocessing

converted = preprocessing.LabelEncoder()
converted.fit(['0','a', 'b', 'c', 'd'])
X["state_holiday"] = converted.transform(X["state_holiday"])

In [3]:
X

,store_ID,day_of_week,nb_customers_on_day,promotion,state_holiday,school_holiday
0,366,4,517,0,0,0
1,394,6,694,0,0,0
2,807,4,970,1,0,0
3,802,2,473,1,0,0
4,726,4,1068,1,0,0
...,...,...,...,...,...,...
640835,409,6,483,0,0,0
640836,97,1,987,1,0,0
640837,987,1,925,0,0,0
640838,1084,4,725,0,0,0


In [4]:
# Standardization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [10]:
# Data split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

kf = KFold(n_splits=5, shuffle=False)
nbSplits = kf.get_n_splits()

acc_snooping = np.zeros((nbSplits,)) #"snooping" refers to a scenario where information from outside the training dataset is inadvertently used to make decisions about the model.
i=0

#We will build the predicted y from the partial predictions on the test of each of the folds
yhat = y.copy()

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dt = KNeighborsClassifier(1)
    dt.fit(X_train,y_train)
    yhat[test_index] = dt.predict(X_test)
    acc_snooping[i] = accuracy_score(yhat[test_index], y_test)

    print(f"R-squared: {(r2_score(y_test, yhat[test_index])):.4f}")
    print(f"MSE: {( mean_squared_error(y_test, yhat[test_index]) / 1_000_000):.4f}")

    i=i+1

print ('Mean accuracy: '+ str(np.mean(acc_snooping)))

R-squared: 0.9105
MSE: 1.3243
R-squared: 0.9125
MSE: 1.3077
R-squared: 0.9121
MSE: 1.3064
R-squared: 0.9124
MSE: 1.2929
R-squared: 0.9144
MSE: 1.2678
Mean accuracy: 0.17044191998002622


In [11]:
#Save the model to disk (it can alternatively be stored in a string)
import pickle
ofname = open('k_fold_v1.pkl', 'wb')
s = pickle.dump(kf,ofname)
ofname.close()
print (s)

None


In [12]:
ofname = open('k_fold_v1.pkl','rb') #Open in binary format. You never know how it was saved.
kf_import = pickle.load(ofname)

In [13]:
y_pred_import = kf_import.predict(X_test)

AttributeError: 'KFold' object has no attribute 'predict'